In [21]:
import pandas as pd

Parto dal dataset originale e prendo solamente i dati rilevanti al progetto, cioè quelli dei passi e quelli legati al sonno


In [22]:
df = pd.read_csv('../data/data.csv.bz2', names=["id", "uuid", "date", "type", "value"])

In [23]:
df = df[(df.type.between(10,19)) | (df.type==1)].drop(columns=["id"])
df.head()

,uuid,date,type,value
0,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,1,10131.39
6,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,10,3.89
7,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,11,22.23
8,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,12,2.93
9,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,13,2.00


Ora devo portarlo da questa forma sparsa a quella tabellare che sarà più facile da utilizzare. Attenzione, se si usa il dataset completo le operazioni sotto richiederanno molta RAM.

In [24]:
names={
 1: "steps",
 10: 'sleepduration',
 11: 'bedin',
 12: 'bedout',
 13: 'nbawake',
 14: 'awakeduration',
 15: 'time to sleep',
 16: 'time to wake up',
 17: 'lightduration',
 18: 'remduration',
 19: 'deepduration'}

In [25]:
for col in names.values(): # Aggiungo una colonna per ognuna delle feature rilevanti
    df[col]=None

In [26]:
for n,col in names.items(): # Su ognuna delle righe copio il valore dal campo "value" a quello della feature corrispondente al tipo
    df[col]=df.where(df.type==n)["value"]
df.head()

,uuid,date,type,value,steps,sleepduration,bedin,bedout,nbawake,awakeduration,time to sleep,time to wake up,lightduration,remduration,deepduration
0,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,1,10131.39,10131.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,10,3.89,NaN,3.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,11,22.23,NaN,NaN,22.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,12,2.93,NaN,NaN,NaN,2.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,13,2.00,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df = df.groupby(["uuid","date"]).max().drop(columns=["type","value"]) # "condenso" le righe relative alla stessa data e allo stesso utente, facendone una sola che contiene tutti i dati. Sono orgoglioso di aver trovato da solo questo modo molto efficiente per ottenere il dataset in forma tabellare densa
df.head()

steps  sleepduration  \
uuid                                     date                                 
1003e58667235e01b49008155604980b3900b00e 2016-07-31  3049.04            NaN   
                                         2016-08-01  4309.81          11.31   
                                         2016-08-02   255.11            NaN   
                                         2016-08-03  5118.07            NaN   
                                         2016-08-04  2228.88          12.90   

                                                     bedin  bedout  nbawake  \
uuid                                     date                                 
1003e58667235e01b49008155604980b3900b00e 2016-07-31    NaN     NaN      NaN   
                                         2016-08-01  22.11    9.86     2.02   
                                         2016-08-02    NaN     NaN      NaN   
                                         2016-08-03    NaN     NaN      NaN   
                                         2016-08-04  20.23    9.79     5.04   

                                                     awakeduration  \
uuid                                     date                        
1003e58667235e01b49008155604980b3900b00e 2016-07-31            NaN   
                                         2016-08-01           0.53   
                                         2016-08-02            NaN   
                                         2016-08-03            NaN   
                                         2016-08-04           0.90   

                                                     time to sleep  \
uuid                                     date                        
1003e58667235e01b49008155604980b3900b00e 2016-07-31            NaN   
                                         2016-08-01           0.15   
                                         2016-08-02            NaN   
                                         2016-08-03            NaN   
                                         2016-08-04           0.10   

                                                     time to wake up  \
uuid                                     date                          
1003e58667235e01b49008155604980b3900b00e 2016-07-31              NaN   
                                         2016-08-01              0.0   
                                         2016-08-02              NaN   
                                         2016-08-03              NaN   
                                         2016-08-04              0.0   

                                                     lightduration  \
uuid                                     date                        
1003e58667235e01b49008155604980b3900b00e 2016-07-31            NaN   
                                         2016-08-01           7.38   
                                         2016-08-02            NaN   
                                         2016-08-03            NaN   
                                         2016-08-04           7.91   

                                                     remduration  deepduration  
uuid                                     date                                   
1003e58667235e01b49008155604980b3900b00e 2016-07-31          NaN           NaN  
                                         2016-08-01          NaN          3.99  
                                         2016-08-02          NaN           NaN  
                                         2016-08-03          NaN           NaN  
                                         2016-08-04          NaN          4.94

In [28]:
df = df.rename_axis(["uuid","date"]).reset_index() # Per un csv più pulito, infatti il groupby aggiunge una colonna che non voglio
df.head()

,uuid,date,steps,sleepduration,bedin,bedout,nbawake,awakeduration,time to sleep,time to wake up,lightduration,remduration,deepduration
0,1003e58667235e01b49008155604980b3900b00e,2016-07-31,3049.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003e58667235e01b49008155604980b3900b00e,2016-08-01,4309.81,11.31,22.11,9.86,2.02,0.53,0.15,0.0,7.38,NaN,3.99
2,1003e58667235e01b49008155604980b3900b00e,2016-08-02,255.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003e58667235e01b49008155604980b3900b00e,2016-08-03,5118.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003e58667235e01b49008155604980b3900b00e,2016-08-04,2228.88,12.90,20.23,9.79,5.04,0.90,0.10,0.0,7.91,NaN,4.94


In [29]:
df.to_csv('../data/dati.csv.gz',compression='gzip',index=False)

Visto che i passi mi serviranno praticamente sempre in forma normalizzata, mi salvo anche la versione del dataset con inclusa la colonna dei passi normalizzati

In [30]:
from sklearn.preprocessing import MinMaxScaler

def scale(x):
 x["user_minmax_steps"]=MinMaxScaler().fit_transform(x[["steps"]]) # Lo scaling è fatto sulla base dell'utente
 return x

df=df.groupby("uuid").apply(scale,include_groups=False)
df.head()

date    steps  \
uuid                                                              
1003e58667235e01b49008155604980b3900b00e 0  2016-07-31  3049.04   
                                         1  2016-08-01  4309.81   
                                         2  2016-08-02   255.11   
                                         3  2016-08-03  5118.07   
                                         4  2016-08-04  2228.88   

                                            sleepduration  bedin  bedout  \
uuid                                                                       
1003e58667235e01b49008155604980b3900b00e 0            NaN    NaN     NaN   
                                         1          11.31  22.11    9.86   
                                         2            NaN    NaN     NaN   
                                         3            NaN    NaN     NaN   
                                         4          12.90  20.23    9.79   

                                            nbawake  awakeduration  \
uuid                                                                 
1003e58667235e01b49008155604980b3900b00e 0      NaN            NaN   
                                         1     2.02           0.53   
                                         2      NaN            NaN   
                                         3      NaN            NaN   
                                         4     5.04           0.90   

                                            time to sleep  time to wake up  \
uuid                                                                         
1003e58667235e01b49008155604980b3900b00e 0            NaN              NaN   
                                         1           0.15              0.0   
                                         2            NaN              NaN   
                                         3            NaN              NaN   
                                         4           0.10              0.0   

                                            lightduration  remduration  \
uuid                                                                     
1003e58667235e01b49008155604980b3900b00e 0            NaN          NaN   
                                         1           7.38          NaN   
                                         2            NaN          NaN   
                                         3            NaN          NaN   
                                         4           7.91          NaN   

                                            deepduration  user_minmax_steps  
uuid                                                                         
1003e58667235e01b49008155604980b3900b00e 0           NaN           0.169272  
                                         1          3.99           0.239404  
                                         2           NaN           0.013856  
                                         3           NaN           0.284364  
                                         4          4.94           0.123650

In [31]:
df = df.rename_axis(["uuid","id"]).reset_index().drop(columns=["id"]) # Per un csv più pulito, infatti il groupby aggiunge una colonna che non voglio
df.head()

,uuid,date,steps,sleepduration,bedin,bedout,nbawake,awakeduration,time to sleep,time to wake up,lightduration,remduration,deepduration,user_minmax_steps
0,1003e58667235e01b49008155604980b3900b00e,2016-07-31,3049.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.169272
1,1003e58667235e01b49008155604980b3900b00e,2016-08-01,4309.81,11.31,22.11,9.86,2.02,0.53,0.15,0.0,7.38,NaN,3.99,0.239404
2,1003e58667235e01b49008155604980b3900b00e,2016-08-02,255.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013856
3,1003e58667235e01b49008155604980b3900b00e,2016-08-03,5118.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.284364
4,1003e58667235e01b49008155604980b3900b00e,2016-08-04,2228.88,12.90,20.23,9.79,5.04,0.90,0.10,0.0,7.91,NaN,4.94,0.123650


In [32]:
df.to_csv('../data/scalati.csv.gz',compression='gzip',index=False)